In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
driver = webdriver.Chrome(service=service)

# Yes24 페이지로 이동 (로그인 및 쿠키 추가 생략)
driver.get('http://ticket.yes24.com/Pages/English/Sale/FnPerfSaleProcess.aspx?IdPerf=51057/')

# 메인 창 핸들 저장
main_window = driver.current_window_handle

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리
handle_alert()

# 좌석 선택 버튼 클릭 (해당 부분을 실제 ID나 경로로 대체)
try:
    seat_selection_button = driver.find_element(By.XPATH, "//img[@alt='좌석 선택완료']")  # 이미지의 alt 속성
    seat_selection_button.click()

    # 새로운 창이 열릴 때까지 대기
    time.sleep(2)

    # 모든 창의 핸들 가져오기
    all_windows = driver.window_handles

    # 팝업 창으로 전환
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("팝업 창으로 전환 성공")

    # 팝업 창에서 좌석 선택 (좌석이 없으면 대기 및 재시도)
    try:
        seats = driver.find_elements(By.CLASS_NAME, 's9')  # 빈 좌석의 클래스 이름이 's9'인 경우
        if seats:
            seats[0].click()  # 첫 번째 빈 좌석 선택
        else:
            print("빈 좌석을 찾을 수 없습니다.")
    except NoSuchElementException:
        print("좌석 선택 버튼을 찾을 수 없습니다.")
    
    # 좌석 선택 완료 후, 확인 버튼 클릭
    driver.execute_script("ChoiceEnd();")  # 자바스크립트 함수 호출

    # 다시 메인 창으로 전환
    driver.switch_to.window(main_window)
    print("메인 창으로 전환 성공")

except NoSuchElementException:
    print("좌석 선택 버튼을 찾을 수 없습니다.")
finally:
    # 창 닫지 않고 유지
    print("작업 완료, 브라우저를 닫지 않고 유지합니다.")
    # driver.quit()  # 창을 닫지 않으려면 주석 처리

좌석 선택 버튼을 찾을 수 없습니다.
작업 완료, 브라우저를 닫지 않고 유지합니다.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import json
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
driver = webdriver.Chrome(service=service)

# Yes24 페이지로 이동 (로그인된 세션 유지 없이 쿠키 추가 전)
driver.get('https://ticket.yes24.com/')

# 쿠키 불러오기
with open('cookies.json', 'r') as cookies_file:
    cookies = json.load(cookies_file)
    for cookie in cookies:
        if 'sameSite' in cookie:
            del cookie['sameSite']  # sameSite 속성 제거
        driver.add_cookie(cookie)

# 쿠키를 추가한 후 새로고침
driver.refresh()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리
handle_alert()

# 로그인된 상태로 예매 페이지로 이동
driver.get('http://ticket.yes24.com/Pages/Perf/Sale/PerfSaleProcess.aspx?IdPerf=51026&IdTime=1313657')

# 팝업 처리
handle_alert()

# 지속적인 좌석 모니터링 설정
monitoring_interval = 10  # 좌석 모니터링 주기 (초)
max_attempts = 30  # 최대 시도 횟수

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            # 빈 좌석 검색
            seats = driver.find_elements(By.CLASS_NAME, 's9')  # 빈 좌석 클래스가 's9'인 경우
            if seats:
                print(f"{len(seats)}개의 빈 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 빈 좌석 선택
                print("좌석을 선택했습니다.")
                
                # 좌석 선택 후 확인 버튼 클릭
                driver.execute_script("ChoiceEnd();")  # 자바스크립트 함수 호출
                return True  # 성공적으로 좌석 선택
            else:
                print(f"빈 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchElementException:
            print("좌석 선택 시 문제가 발생했습니다. 다시 시도합니다.")

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# 결제 단계는 수동으로 처리하기 위해 브라우저 닫지 않음
print("작업 완료, 브라우저를 닫지 않고 유지합니다.")
# driver.quit()  # 수동 결제를 위해 브라우저는 닫지 않음

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import json
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
driver = webdriver.Chrome(service=service)

# Yes24 페이지로 이동 (쿠키가 없는 상태)
driver.get('https://ticket.yes24.com/')

# 저장된 쿠키 불러오기
with open('cookies.json', 'r') as cookies_file:
    cookies = json.load(cookies_file)
    for cookie in cookies:
        if 'sameSite' in cookie:
            del cookie['sameSite']
        driver.add_cookie(cookie)

# 쿠키 추가 후 페이지 새로고침
driver.refresh()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 로그인된 상태로 예매 페이지로 이동
driver.get('https://ticket.yes24.com/Pages/Perf/Sale/PerfSaleProcess.aspx?IdPerf=51026&IdTime=1313657')

# 팝업 창이 뜨면 처리
handle_alert()

# 예매 버튼 찾기
try:
    seat_selection_button = driver.find_element(By.XPATH, "//img[@alt='좌석 선택완료']")  # 예매 버튼 이미지 확인
    seat_selection_button.click()
    print("예매 버튼을 클릭했습니다.")

    # 팝업 처리 반복
    handle_alert()

    # 예매 과정 진행
    time.sleep(2)  # 페이지가 로드되는 시간을 충분히 대기
except NoSuchElementException:
    print("예매 버튼을 찾을 수 없습니다. 다시 시도합니다.")
    driver.quit()

# 좌석 모니터링 로직
def monitor_and_select_seat():
    attempts = 0
    monitoring_interval = 10
    max_attempts = 30
    
    while attempts < max_attempts:
        try:
            seats = driver.find_elements(By.CLASS_NAME, 's9')  # 빈 좌석 클래스가 's9'인 경우
            if seats:
                print(f"{len(seats)}개의 빈 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 빈 좌석 선택
                driver.execute_script("ChoiceEnd();")  # 좌석 선택 완료 자바스크립트 호출
                return True
            else:
                print(f"빈 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchElementException:
            print("좌석 선택 중 문제가 발생했습니다. 다시 시도합니다.")

        time.sleep(monitoring_interval)
        attempts += 1
    
    return False

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다.")

# 브라우저 유지
print("작업 완료, 브라우저를 닫지 않고 유지합니다.")

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import json
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
try:
    driver = webdriver.Chrome(service=service)
    print("ChromeDriver 연결 성공")
except Exception as e:
    print(f"ChromeDriver 연결 실패: {e}")
    exit()

# Yes24 메인 페이지로 이동
driver.get('https://ticket.yes24.com/')

# 저장된 쿠키 불러오기
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 쿠키 추가 후 페이지 새로고침
driver.refresh()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 로그인된 상태로 예매 페이지로 이동
driver.get('https://ticket.yes24.com/Pages/Perf/Sale/PerfSaleProcess.aspx?IdPerf=51026&IdTime=1313657')

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 예매 버튼 클릭 (첫 번째 화면에서 빨간 예매하기 버튼 클릭)
try:
    reserve_button = driver.find_element(By.XPATH, "//a[contains(text(), '예매하기')]")
    reserve_button.click()
    print("예매 버튼을 클릭했습니다.")

    # 예매 버튼 클릭 후 팝업 처리 대기
    time.sleep(2)
    handle_alert()

    # 페이지 로드 대기
    time.sleep(3)  # 페이지 로딩 시간 대기

    # 날짜와 시간을 선택 (적절한 옵션 선택 추가 필요)
    date_button = driver.find_element(By.XPATH, "//select[@id='dateSelect']")
    time_button = driver.find_element(By.XPATH, "//select[@id='timeSelect']")

    date_button.click()  # 날짜 선택
    time.sleep(1)
    time_button.click()  # 시간 선택

    # 다음 단계 버튼 클릭
    next_button = driver.find_element(By.XPATH, "//a[@id='nextButtonId']")  # 실제 ID로 교체 필요
    next_button.click()

    # 좌석 선택 페이지로 이동 후 좌석 선택
    time.sleep(3)  # 페이지 로딩 시간 대기
    seats = driver.find_elements(By.CLASS_NAME, 's9')  # 빈 좌석 클래스가 's9'인 경우
    if seats:
        print(f"{len(seats)}개의 빈 좌석을 찾았습니다.")
        seats[0].click()  # 첫 번째 빈 좌석 선택
        driver.execute_script("ChoiceEnd();")  # 자바스크립트 함수 호출로 좌석 선택 완료
    else:
        print("빈 좌석을 찾을 수 없습니다.")
except NoSuchElementException as e:
    print(f"오류 발생: {e}")
    driver.quit()

# 결제 단계는 수동으로 처리
print("작업 완료, 브라우저를 닫지 않고 유지합니다.")


ChromeDriver 연결 성공
쿠키 로드 성공
팝업 텍스트: 예매버튼을 이용하여 예매 부탁 드립니다.
오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(text(), '예매하기')]"}
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7FAEEB632+29090]
	(No symbol) [0x00007FF7FAE5E6E9]
	(No symbol) [0x00007FF7FAD1B1CA]
	(No symbol) [0x00007FF7FAD6EFD7]
	(No symbol) [0x00007FF7FAD6F22C]
	(No symbol) [0x00007FF7FADB97F7]
	(No symbol) [0x00007FF7FAD9672F]
	(No symbol) [0x00007FF7FADB65D9]
	(No symbol) [0x00007FF7FAD96493]
	(No symbol) [0x00007FF7FAD609B1]
	(No symbol) [0x00007FF7FAD61B11]
	GetHandleVerifier [0x00007FF7FB20881D+3294093]
	GetHandleVerifier [0x00007FF7FB254403+3604339]
	GetHandleVerifier [0x00007FF7FB24A2C7+3563063]
	GetHandleVerifier [0x00007FF7FAFA6F16+797318]
	(No symbol) [0x00007FF7FAE6986F]
	(No symbol)

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import json
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")  # User-Agent 설정
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Selenium 탐지 방지
chrome_options.add_experimental_option("useAutomationExtension", False)

# Chrome 실행
try:
    driver = webdriver.Chrome(service=service)
    print("ChromeDriver 연결 성공")
except Exception as e:
    print(f"ChromeDriver 연결 실패: {e}")
    exit()

# Yes24 메인 페이지로 이동
driver.get('https://ticket.yes24.com/')

# 저장된 쿠키 불러오기
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 쿠키 추가 후 페이지 새로고침
driver.refresh()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 쿠키와 팝업 처리 후 티켓팅 페이지로 이동
driver.get('https://ticket.yes24.com/Pages/Perf/Sale/PerfSaleProcess.aspx?IdPerf=51057')  # 티켓팅 페이지 주소 삽입
handle_alert()

# 예매 버튼을 클릭하는 로직
try:
    # 페이지가 완전히 로드될 때까지 대기 (최대 10초 대기)
    wait = WebDriverWait(driver, 10)
    
    # 예매하기 버튼이 로드되었는지 확인하고 클릭
    reserve_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '예매하기')]")))
    reserve_button.click()
    print("예매 버튼을 클릭했습니다.")
    
    # 예매 버튼 클릭 후 팝업 처리 대기
    time.sleep(2)
    handle_alert()

except TimeoutException:
    print("예매 버튼을 찾을 수 없습니다. 다시 시도합니다.")
except NoSuchElementException as e:
    print(f"오류 발생: {e}")
finally:
    # 브라우저 유지
    print("작업 완료, 브라우저를 닫지 않고 유지합니다.")
    input("브라우저를 닫지 않으려면 엔터를 누르세요...")  # 사용자가 수동으로 엔터를 누를 때까지 브라우저 유지

# 브라우저 유지 (자동 종료 방지)
# driver.quit()  # 이 부분은 주석 처리하여 브라우저가 자동으로 닫히지 않도록 함


ChromeDriver 연결 성공
쿠키 로드 성공
팝업 텍스트: 정상적인 접근이 아닙니다(NF_Null).
예매 버튼을 찾을 수 없습니다. 다시 시도합니다.
작업 완료, 브라우저를 닫지 않고 유지합니다.


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Yes24 메인 페이지로 이동
driver.get('https://ticket.yes24.com/')

# 저장된 쿠키 불러오기
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

time.sleep(2)

# Yes24 티켓팅 페이지로 이동
driver.get('http://ticket.yes24.com/Pages/Perf/Sale/PerfSaleProcess.aspx?IdPerf=51057')

# 날짜 선택 (2024년 10월 20일)
try:
    date_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, '2024-10-20'))
    )
    date_element.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회 13:00)
try:
    session_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//li[contains(text(), '13시 00분')]"))
    )
    session_element.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 오류: {e}")

# 좌석 선택 및 이후 단계 처리 로직 추가 가능
# 예시로 좌석을 자동으로 선택한 후 다음 단계로 넘어가는 코드
try:
    seats = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 's9'))  # 빈 좌석 클래스가 's9'인 경우
    )
    if seats:
        seats[0].click()  # 첫 번째 빈 좌석 선택
        print("좌석 선택 완료")
    
    confirm_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'confirm_button_id'))  # 좌석 선택 후 "확인" 버튼
    )
    confirm_button.click()
    print("확인 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

# 결제 단계로 이동
try:
    next_step_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnNextStep'))
    )
    next_step_button.click()
    print("다음 단계로 이동")
except Exception as e:
    print(f"다음 단계 이동 오류: {e}")

# 브라우저 유지
print("작업 완료, 브라우저를 닫지 않고 유지합니다.")


쿠키 로드 성공
날짜 선택 오류: Alert Text: 정상적인 접근이 아닙니다(NF_Null).
Message: unexpected alert open: {Alert text : 정상적인 접근이 아닙니다(NF_Null).}
  (Session info: chrome=129.0.6668.90)
Stacktrace:
	GetHandleVerifier [0x00007FF7FAEEB632+29090]
	(No symbol) [0x00007FF7FAE5E6E9]
	(No symbol) [0x00007FF7FAD1B1CA]
	(No symbol) [0x00007FF7FADB7196]
	(No symbol) [0x00007FF7FAD96493]
	(No symbol) [0x00007FF7FAD609B1]
	(No symbol) [0x00007FF7FAD61B11]
	GetHandleVerifier [0x00007FF7FB20881D+3294093]
	GetHandleVerifier [0x00007FF7FB254403+3604339]
	GetHandleVerifier [0x00007FF7FB24A2C7+3563063]
	GetHandleVerifier [0x00007FF7FAFA6F16+797318]
	(No symbol) [0x00007FF7FAE6986F]
	(No symbol) [0x00007FF7FAE65454]
	(No symbol) [0x00007FF7FAE655E0]
	(No symbol) [0x00007FF7FAE54A7F]
	BaseThreadInitThunk [0x00007FF89441257D+29]
	RtlUserThreadStart [0x00007FF89562AF28+40]

회차 선택 오류: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FAEEB632+29090]
	(No symbol) [0x00007FF7FAE5E6E9]
	(No symbol) [0x00007FF7FAD1B1CA]
	(No symbo